In [ ]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import re

# =========================
# 路径设置
# =========================
npy_dir = "../XCO2_prediction_full"
out_dir = "../XCO2_prediction_full_nc"
os.makedirs(out_dir, exist_ok=True)

# 文件名匹配
pattern = re.compile(r"monthly_xco2_full_(\d{4})_(\d{2})\.npy")

# =========================
# 批量处理
# =========================
for fname in sorted(os.listdir(npy_dir)):
    match = pattern.match(fname)
    if match is None:
        continue

    year, month = match.groups()
    print(f"\n=== Processing {year}-{month} ===")

    fpath = os.path.join(npy_dir, fname)

    # -------------------------
    # 1. 读取 structured npy
    # -------------------------
    data = np.load(fpath)

    # 转 DataFrame（最稳妥）
    df = pd.DataFrame(data)

    # 确保 time 是 datetime
    df["time"] = pd.to_datetime(df["time"])

    # -------------------------
    # 2. 转 xarray（pivot）
    # -------------------------
    ds = (
        df
        .set_index(["time", "lat", "lon"])
        .to_xarray()
    )

    # 只保留你需要的变量
    ds = ds[["xco2_pred"]]

    # -------------------------
    # 3. 添加变量 & 全局属性
    # -------------------------
    ds["xco2_pred"].attrs.update(
        units="ppm",
        long_name="Daily column-averaged CO2 dry air mole fraction (predicted)"
    )

    ds.attrs.update(
        title="Daily XCO2 prediction",
        source="Converted from structured npy",
        year=int(year),
        month=int(month)
    )

    # -------------------------
    # 4. 写 NetCDF（压缩）
    # -------------------------
    out_nc = os.path.join(
        out_dir,
        f"xco2_pred_daily_{year}_{month}.nc"
    )

    encoding = {
        "xco2_pred": {
            "zlib": True,
            "complevel": 4,
            "dtype": "float32"
        }
    }

    ds.to_netcdf(out_nc, encoding=encoding)
    ds.close()

    # 显式释放内存（很重要）
    del df, ds, data

    print(f"Saved: {out_nc}")

print("\n✅ All months processed successfully.")



=== Processing 2019-01 ===
